In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
#data describe
YouTube_revenue_analyse=pd.read_csv("C:/Users/user/Desktop/VS exc/VS Code/youtube_ad_revenue_dataset.csv")
print(YouTube_revenue_analyse)

YouTube_revenue_analyse.head()

rows_count,column_count=YouTube_revenue_analyse.shape
YouTube_revenue_analyse.info()
YouTube_revenue_analyse.describe()
YouTube_revenue_analyse.isnull().sum()

        video_id     date  views   likes  comments  watch_time_minutes  \
0       vid_3092  50:41.0   9936  1221.0     320.0         26497.21418   
1       vid_3459  50:41.0  10017   642.0     346.0         15209.74744   
2       vid_4784  50:41.0  10097  1979.0     187.0         57332.65850   
3       vid_4078  50:41.0  10034  1191.0     242.0         31334.51777   
4       vid_3522  50:41.0   9889  1858.0     477.0         15665.66643   
...          ...      ...    ...     ...       ...                 ...   
122395  vid_2902  50:41.0   9853  1673.0     147.0         42075.70489   
122396  vid_3890  50:41.0  10128  1709.0      63.0         57563.70304   
122397  vid_3934  50:41.0  10267   700.0       NaN         27549.71466   
122398  vid_4260  50:41.0  10240  1616.0     106.0         56967.38438   
122399  vid_1056  50:41.0   9931   770.0       NaN         38466.83714   

        video_length_minutes  subscribers       category   device country  \
0                   2.862137      

video_id                   0
date                       0
views                      0
likes                   6117
comments                6112
watch_time_minutes      6105
video_length_minutes       0
subscribers                0
category                   0
device                     0
country                    0
ad_revenue_usd             0
dtype: int64

In [3]:
#data cleaning
YouTube_revenue_analyse=YouTube_revenue_analyse.drop(["video_id","date"],axis=1)
YouTube_revenue_analyse.info()

YouTube_revenue_analyse["likes"].fillna(YouTube_revenue_analyse["likes"].median(), inplace=True)
print(YouTube_revenue_analyse["likes"])
YouTube_revenue_analyse.info()

YouTube_revenue_analyse["comments"].fillna(YouTube_revenue_analyse["comments"].mode()[0],inplace=True)
print(YouTube_revenue_analyse["comments"])
YouTube_revenue_analyse.info()

YouTube_revenue_analyse["watch_time_minutes"]=YouTube_revenue_analyse["watch_time_minutes"].ffill()
print(YouTube_revenue_analyse["watch_time_minutes"])
YouTube_revenue_analyse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   views                 122400 non-null  int64  
 1   likes                 116283 non-null  float64
 2   comments              116288 non-null  float64
 3   watch_time_minutes    116295 non-null  float64
 4   video_length_minutes  122400 non-null  float64
 5   subscribers           122400 non-null  int64  
 6   category              122400 non-null  object 
 7   device                122400 non-null  object 
 8   country               122400 non-null  object 
 9   ad_revenue_usd        122400 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 9.3+ MB
0         1221.0
1          642.0
2         1979.0
3         1191.0
4         1858.0
           ...  
122395    1673.0
122396    1709.0
122397     700.0
122398    1616.0
122399     770.0
Name: likes, Len

C:\Users\user\AppData\Local\Temp\ipykernel_14496\1912703623.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  YouTube_revenue_analyse["likes"].fillna(YouTube_revenue_analyse["likes"].median(), inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14496\1912703623.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [4]:
#data labling
le = LabelEncoder()
YouTube_revenue_analyse['country_encoded'] = le.fit_transform(YouTube_revenue_analyse['country'])
YouTube_revenue_analyse['category_encoded'] = le.fit_transform(YouTube_revenue_analyse['category'])
YouTube_revenue_analyse['device_encoded'] = le.fit_transform(YouTube_revenue_analyse['device'])
YouTube_revenue_analyse.head()


,views,likes,comments,watch_time_minutes,video_length_minutes,subscribers,category,device,country,ad_revenue_usd,country_encoded,category_encoded,device_encoded
0,9936,1221.0,320.0,26497.21418,2.862137,228086,Entertainment,TV,IN,203.178237,3,1,2
1,10017,642.0,346.0,15209.74744,23.738069,736015,Gaming,Tablet,CA,140.880508,1,2,3
2,10097,1979.0,187.0,57332.65850,26.200634,240534,Education,TV,CA,360.134007,1,0,2
3,10034,1191.0,242.0,31334.51777,11.770340,434482,Entertainment,Mobile,UK,224.638261,4,1,1
4,9889,1858.0,477.0,15665.66643,6.635854,42030,Education,Mobile,CA,165.514388,1,0,1


In [5]:
YouTube_revenue_analyse.drop(columns=['country'], inplace=True)
YouTube_revenue_analyse.drop(columns=['category'], inplace=True)
YouTube_revenue_analyse.drop(columns=['device'], inplace=True)
YouTube_revenue_analyse.tail()

,views,likes,comments,watch_time_minutes,video_length_minutes,subscribers,ad_revenue_usd,country_encoded,category_encoded,device_encoded
122395,9853,1673.0,147.0,42075.70489,25.490195,210818,280.986396,5,0,3
122396,10128,1709.0,63.0,57563.70304,16.229133,878860,354.612981,4,4,0
122397,10267,700.0,158.0,27549.71466,23.822365,576756,203.643106,1,5,3
122398,10240,1616.0,106.0,56967.38438,7.753099,585138,351.525811,4,4,1
122399,9931,770.0,158.0,38466.83714,27.305703,517587,253.842824,1,5,2


In [6]:
# Train and Test Split
X = YouTube_revenue_analyse.drop(columns = ['ad_revenue_usd'])
y = YouTube_revenue_analyse['ad_revenue_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 0.10 - 0.25
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((97920, 9), (24480, 9), (97920,), (24480,))

In [7]:
# Scaling of the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(X_test)

[[-0.5883677   1.51159336 -0.69604891 ...  0.30073411 -1.46134979
  -0.44793684]
 [ 0.6919651   1.45034003 -0.45555395 ...  0.88839892  0.87698181
   0.44538242]
 [-0.6283781  -0.74490009  0.73916295 ... -0.87459552  0.29239891
  -0.44793684]
 ...
 [-0.4783391  -1.35348152  0.11853079 ...  0.88839892 -1.46134979
   0.44538242]
 [ 0.7619833   0.02768216 -0.75035423 ...  0.88839892  0.29239891
   1.33870167]
 [-0.2682845   1.3456166   0.32799414 ...  1.47606373  0.29239891
   1.33870167]]
[[-1.3385627  -1.07882951  0.18835191 ... -0.28693071  0.29239891
  -1.3412561 ]
 [-0.7484093   0.60662488  1.7321744  ...  0.30073411 -0.87676689
   0.44538242]
 [-1.4385887  -1.0630222  -0.88999646 ... -0.87459552  1.46156471
   0.44538242]
 ...
 [-0.6283781   0.18180343 -0.18402739 ... -0.28693071  0.87698181
   0.44538242]
 [-0.9684665  -0.58287517  1.03396322 ...  0.88839892 -0.29218399
   0.44538242]
 [ 1.3221289  -0.74094827  1.27445818 ... -0.28693071 -1.46134979
   0.44538242]]


In [8]:
# Model Training
model = LinearRegression()
model.fit(X_train, y_train)
print(model.coef_)
print(model.intercept_)
# Testing of the dataset
y_prediction = model.predict(X_test)
y_actual = y_test
y_prediction

# Evaluation

mse = mean_squared_error(y_actual, y_prediction)
mae = mean_absolute_error(y_actual, y_prediction)
r2 = r2_score(y_actual, y_prediction)
rmse = np.sqrt(mse)

print("mae=",mae)
print("r2=",r2 * 100)
print("mse=",mse)
print("rmse=",rmse)


[ 7.28191767e-01  8.69866006e+00  2.00217093e+00  5.81867843e+01
  5.27784286e-02  1.66439685e-02  2.55321211e-02 -4.48503259e-02
 -2.55447311e-03]
252.45999049757148
mae= 6.253611389760955
r2= 90.39535776951493
mse= 368.2834117555228
rmse= 19.19071160107209


In [ ]:
#Ridge Regression
# Train and Test Split
X = YouTube_revenue_analyse.drop(columns = ['ad_revenue_usd'])
y = YouTube_revenue_analyse['ad_revenue_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 0.10 - 0.25
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Scaling of the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(X_test)

ridge_model = Ridge(alpha=0.0001) 
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)

ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)
ridge_rmse = np.sqrt(ridge_mse)
ridge_r2 = r2_score(y_test, ridge_pred)

print("MAE:", ridge_mae)
print("MSE:", ridge_mse)
print("RMSE:", ridge_rmse)
print("R²:", ridge_r2*100)



[[-0.5883677   1.51159336 -0.69604891 ...  0.30073411 -1.46134979
  -0.44793684]
 [ 0.6919651   1.45034003 -0.45555395 ...  0.88839892  0.87698181
   0.44538242]
 [-0.6283781  -0.74490009  0.73916295 ... -0.87459552  0.29239891
  -0.44793684]
 ...
 [-0.4783391  -1.35348152  0.11853079 ...  0.88839892 -1.46134979
   0.44538242]
 [ 0.7619833   0.02768216 -0.75035423 ...  0.88839892  0.29239891
   1.33870167]
 [-0.2682845   1.3456166   0.32799414 ...  1.47606373  0.29239891
   1.33870167]]
[[-1.3385627  -1.07882951  0.18835191 ... -0.28693071  0.29239891
  -1.3412561 ]
 [-0.7484093   0.60662488  1.7321744  ...  0.30073411 -0.87676689
   0.44538242]
 [-1.4385887  -1.0630222  -0.88999646 ... -0.87459552  1.46156471
   0.44538242]
 ...
 [-0.6283781   0.18180343 -0.18402739 ... -0.28693071  0.87698181
   0.44538242]
 [-0.9684665  -0.58287517  1.03396322 ...  0.88839892 -0.29218399
   0.44538242]
 [ 1.3221289  -0.74094827  1.27445818 ... -0.28693071 -1.46134979
   0.44538242]]
MAE: 6.253615860

In [26]:
#Lasso Regression

# Train and Test Split
X = YouTube_revenue_analyse.drop(columns = ['ad_revenue_usd'])
y = YouTube_revenue_analyse['ad_revenue_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 0.10 - 0.25
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Scaling of the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(X_test)

ridge_model = Ridge(alpha=0.01) 
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)

lasso_model = Lasso(alpha=0.1, max_iter=10000)
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)

lasso_mae = mean_absolute_error(y_test, lasso_pred)
lasso_mse = mean_squared_error(y_test, lasso_pred)
lasso_rmse = np.sqrt(lasso_mse)
lasso_r2 = r2_score(y_test, lasso_pred)

print("MAE:", lasso_mae)
print("MSE:", lasso_mse)
print("RMSE:", lasso_rmse)
print("R²:", lasso_r2*100)

[[-0.5883677   1.51159336 -0.69604891 ...  0.30073411 -1.46134979
  -0.44793684]
 [ 0.6919651   1.45034003 -0.45555395 ...  0.88839892  0.87698181
   0.44538242]
 [-0.6283781  -0.74490009  0.73916295 ... -0.87459552  0.29239891
  -0.44793684]
 ...
 [-0.4783391  -1.35348152  0.11853079 ...  0.88839892 -1.46134979
   0.44538242]
 [ 0.7619833   0.02768216 -0.75035423 ...  0.88839892  0.29239891
   1.33870167]
 [-0.2682845   1.3456166   0.32799414 ...  1.47606373  0.29239891
   1.33870167]]
[[-1.3385627  -1.07882951  0.18835191 ... -0.28693071  0.29239891
  -1.3412561 ]
 [-0.7484093   0.60662488  1.7321744  ...  0.30073411 -0.87676689
   0.44538242]
 [-1.4385887  -1.0630222  -0.88999646 ... -0.87459552  1.46156471
   0.44538242]
 ...
 [-0.6283781   0.18180343 -0.18402739 ... -0.28693071  0.87698181
   0.44538242]
 [-0.9684665  -0.58287517  1.03396322 ...  0.88839892 -0.29218399
   0.44538242]
 [ 1.3221289  -0.74094827  1.27445818 ... -0.28693071 -1.46134979
   0.44538242]]
MAE: 6.328367548

In [43]:
#Multiple linear regression

X = YouTube_revenue_analyse[["comments", "watch_time_minutes", "likes"]]   
y = YouTube_revenue_analyse["ad_revenue_usd"]                        

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("MULTIPLE LINEAR REGRESSION")
print("Coefficients=", model.coef_)      # slope values
print("Intercept=", model.intercept_)    # intercept called C
print("R2=", r2_score(y_test, y_pred)*100)
print("MSE=", mean_squared_error(y_test, y_pred))
print("MAE=", mean_absolute_error(y_test, y_pred))



MULTIPLE LINEAR REGRESSION
Coefficients= [0.01563121 0.00448103 0.01720854]
Intercept= 61.34083642064232
R2= 90.37992535196867
MSE= 368.8751572103959
MAE= 6.269893840516802


In [ ]:
#checking values r^2 values
#r2 test = r2 train means good fitting data no outliers

X = YouTube_revenue_analyse[["comments", "watch_time_minutes", "likes"]]   
y = YouTube_revenue_analyse["ad_revenue_usd"]    

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R² :", r2_score(y_test, y_test_pred))




Train R²: 0.9035930683378343
Test R² : 0.9052813465406981


In [44]:
#elastic net regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.1, random_state=42)
elastic_model.fit(X_train, y_train)
y_pred = elastic_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2*100)



MAE: 6.269920401700966
MSE: 368.8754530439744
RMSE: 19.206130610926667
R²: 90.37991763678089


In [41]:
#random forest regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=50, max_depth=15, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Random Forest Regression Metrics:")
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2*100)


Random Forest Regression Metrics:
MAE: 7.134774780477405
MSE: 398.2384536327207
RMSE: 19.95591274867478
R²: 89.61414566208319


In [38]:
#Gradient Boosting

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Gradient Boosting Model
gbr = GradientBoostingRegressor(n_estimators=50, max_depth=10, random_state=42)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)

# Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Gradient Boosting Results:")
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2*100)


Gradient Boosting Results:
MAE: 7.016868399046013
MSE: 393.8176457889702
RMSE: 19.844839273447647
R²: 89.72943806014844
